In [2]:
from pandas import Series,DataFrame
import pandas as pd
import os
import math

In [3]:
NAN=-1
data_path="C:/Users/heo/Desktop/Mosquito/mosquito_data/"
data_list = os.listdir(data_path)

In [4]:
weather = pd.read_csv("C:/Users/heo/Desktop/Mosquito/weather_data.csv",encoding = "euc-kr")

In [5]:
del weather['지점']
weather.loc[weather.강수량 == ' ', '강수량'] = 0

weather.columns = ['Date','Hum','Raf','Tmx','Tav','Tmi']
weather['Date'] = pd.to_datetime(weather['Date'])

weather = weather.set_index("Date")
weather['Raf'] = weather['Raf'].astype(float)

In [6]:
def Accumulation(weather, name, threshold):
    accumulation_list=[]
    index_num=0
    for index, row in weather.iterrows():
        if index_num < threshold:
            accumulation_list.append(weather.iloc[0:index_num][name].sum())
        else:
            accumulation_list.append(weather.iloc[index_num - threshold:index_num][name].sum())
        index_num += 1
    weather.insert(0, name+str(threshold), accumulation_list)

In [7]:
name_list=['Tmx','Tmi','Tav','Raf','Hum']
threshold_list=[19,2,16,6,5]
for name,threshold in zip(name_list,threshold_list):
    Accumulation(weather,name,threshold)

In [8]:
accumulation_list = []
index_num = 0
for index, row in weather.iterrows():
    cnt=0
    if index_num < 29:
        for i in range(0,index_num):
            if weather['Raf'][i]>0.0:
                cnt+=1
    else:
        for i in range(index_num-29,index_num):
            if weather['Raf'][i]>0.0:
                cnt+=1
    accumulation_list.append(cnt)
    index_num += 1
weather.insert(2, 'RfD29', accumulation_list)

In [9]:
for i in name_list:
    del weather[i]

In [10]:
def Processing(name,weather):
    mosquito = pd.read_csv(data_path+name,names=['Date','Mosq'])
    mosquito['Date'] = pd.to_datetime(mosquito['Date'])
    mosquito = mosquito.set_index("Date")
    mosquito = mosquito.resample('D').sum()
    mosquito.insert(0, 'Location', name[:-11])
    mosquito.insert(1,"LandUse",name[-10:-9])

    for index, row in mosquito.iterrows():
        if 1000 < row['Mosq'] and row['Mosq']<10000:
            mosquito.set_value(index,'Mosq',1000)

    for index, row in mosquito.iterrows():
        if row['Mosq'] >= 10000:
            month = str(index)[:7]
            mosquito.set_value(index, 'Mosq', int(mosquito[month]['Mosq'].mean()))

        mosquito = mosquito.fillna(NAN)
    for index, row in mosquito.iterrows():
        if row['Mosq'] ==-1:
            month = str(index)[:7]
            value = int(mosquito[month]['Mosq'].mean())
            if value==-1:
                mosquito.set_value(index, 'Mosq', 0)
            else:
                mosquito.set_value(index, 'Mosq', value)

    mosquito = pd.concat([mosquito, weather], axis=1, join="inner")
    return mosquito

In [11]:
concat_list=[]
for name in data_list:
    concat_list.append(Processing(name,weather))
mosquito = pd.concat(concat_list)

In [12]:
mosquito = mosquito[['Location','Hum5','Raf6','RfD29','Tav16','Tmi2','Tmx19','Mosq',"LandUse"]]

In [13]:
level=[]
for row in mosquito['Mosq']:
    if row <=20:
        level.append(1)
    else:
        level.append(math.ceil(math.log(row/10,2)))
mosquito['level']=level
del mosquito['Mosq']

In [14]:
mosquito['Date'] = mosquito.index
mosquito = mosquito.set_index("Location")
mosquito.index.name = "Location"
mosquito = mosquito[['Date','Hum5','Raf6','RfD29','Tav16','Tmi2','Tmx19',"LandUse","level"]]

In [2]:
mosquito.to_csv('result.csv')

NameError: name 'mosquito' is not defined